In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

We used the notebook: https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn as a baseline for this notebook.

# Import the libraries

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
import random 
import os
import cv2
import gc
from tqdm.auto import tqdm
import numpy as np
from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
import datetime as dt


## Get the data

In [ ]:
project_dir = "/kaggle/input/bengaliai-cv19/"

train_data_pre = pd.read_csv(project_dir+"/train.csv")
train_data_pre.shape

## Setting up the data

we took this method from the notebook: https://www.kaggle.com/shawon10/bangla-graphemes-image-processing-deep-cnn

In [ ]:
def resize(df, size=64, need_progress_bar=True):
    resized = {}
    resize_size=64
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])):
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    else:
        for i in range(df.shape[0]):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

# some analysis

In [ ]:
train_labels = train_data_pre[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme']]
y_train = np.array(train_labels.iloc[1:,0:3])
root_dist = np.bincount(y_train[:,0])
vowel_dist = np.bincount(y_train[:,1])
consonant_dist = np.bincount(y_train[:,2])

In [ ]:
import seaborn as sns

plt.rcParams["patch.force_edgecolor"] = True
#ax = sns.distplot(root_dist, kde=False, rug=False, vertical = False)
#plt.xlabel="Class count"
#ax.set_title("Histogram of the distribution of the grapheme root classes")
#ax.set(xlabel='Nr. of samples in a class', ylabel='Nr. of classes')
import matplotlib.pyplot as plt

hist1 = plt.hist(root_dist, color="skyblue")
plt.xlabel('Nr. of samples in a class')
plt.ylabel('Nr. of classes')
plt.title('Histogram of the distribution of the grapheme root classes')
plt.xticks(hist1[1])

plt.show()

df_vowel = pd.DataFrame(
    [[i+1, vowel_dist[i]] for i in range(len(vowel_dist))]
)
df_vowel.columns = ['Class', 'Nr. of samples in a class']


ax = sns.barplot(x="Class", y="Nr. of samples in a class", data=df_vowel, palette="Blues_d")
ax.set_title("Bar chart of the number of samples in the vowel diacritics classes")
plt.show()

df_consonant = pd.DataFrame(
    [[i+1, consonant_dist[i]] for i in range(len(consonant_dist))]
)
df_consonant.columns = ['Class', 'Nr. of samples in a class']

ax = sns.barplot(x="Class", y="Nr. of samples in a class", data=df_consonant,  palette="Blues_d")
ax.set_title("Bar chart of the number of samples in the consonant diacritics classes")
plt.show()

## The model

We switched from three separate models to one multi-output model because of the following reasons:
* Storing such a deep convolutional network needs a lot of RAM, especially with 3 separate models since we had to store three models instead of one. Because of this, our Kaggle notebook exceeded its maximum amount of RAM usage.
* With one network we were able to make the network 'deeper' --we could add more layers with more channels; therefore our accuracy score increased heavily. 

In [ ]:
IMG_SIZE = 64

inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))

model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1))(inputs)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Flatten()(model)
model = Dense(1024, activation = "relu")(model)
model = Dropout(rate=0.3)(model)
dense = Dense(512, activation = "relu")(model)

head_root = Dense(168, activation = 'softmax', name='root')(dense)
head_vowel = Dense(11, activation = 'softmax', name = 'vowel')(dense)
head_consonant = Dense(7, activation = 'softmax', name='consonant')(dense)

model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

In [ ]:
model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

## Train the model

In [ ]:
from sklearn.utils import class_weight
batch_size = 256
epochs = 25
target_dict = {}
class_weight_dict = {}
model_types = ['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']

history_dict = []

for i in range(0,4):
    train_data =  pd.merge(pd.read_feather(f'/kaggle/input/bengaliaicv19feather/train_image_data_{i}.feather') , train_data_pre, on='image_id').drop(['image_id'], axis=1)
    train_labels = train_data[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme']]
    train_data = train_data.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme'], axis=1)
    train_data = resize(train_data)/255
    train_data = train_data.values.reshape(-1, 64, 64, 1)


    for target in model_types:
        y_train_one_hot = train_labels[target]
        y_train_one_hot = pd.get_dummies(y_train_one_hot).values
        target_dict[target] = y_train_one_hot
        class_weight_dict[target] = class_weight.compute_class_weight('balanced',
                                                     np.unique(train_labels[target]),
                                                     train_labels[target])

    #splitting the data into a train and validation set
    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(train_data, target_dict['grapheme_root'], target_dict['vowel_diacritic'], target_dict['consonant_diacritic'], test_size=0.1, random_state=123)


    datagen = MultiOutputDataGenerator(
        rotation_range=10,  
        zoom_range = 0.2, 
        width_shift_range=0.12,  
        height_shift_range=0.12,
        horizontal_flip=True)
    
    datagen.fit(x_train)
    history = model.fit_generator(datagen.flow(x_train, {'root': y_train_root, 'vowel': y_train_vowel, 'consonant': y_train_consonant}, batch_size=batch_size), 
                                                 epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                                  steps_per_epoch=x_train.shape[0] // batch_size, class_weight={'root': class_weight_dict['grapheme_root'], 
                                                                                                'vowel': class_weight_dict['vowel_diacritic'], 
                                                                                                'consonant': class_weight_dict['consonant_diacritic']})
    history_dict.append(history)
    model.save('model.h5')
    del x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant, train_data, train_labels
    gc.collect()

## Plot the training process

In [ ]:
print(history.history.keys())

In [ ]:
dataset = 0
for history in history_dict:
    
    #accuracy
    output_names = ['root', 'vowel', 'consonant']
    plt.figure()
    for i in range(3):
        plt.plot(np.arange(0, epochs), history.history[output_names[i]+'_accuracy'], label='train_accuracy')
        plt.plot(np.arange(0, epochs), history.history['val_'+output_names[i]+'_accuracy'], label='val_accuracy')
        plt.title('Accuracy of '+model_types[i]+' for training dataset '+ str(dataset))
        plt.xlabel('Epoch #')
        plt.ylabel('Accuracy')
        plt.legend(loc='lower right')
        plt.show()
        print('Validation accuracy for {} is: {}'.format(model_types[i], max(history.history['val_'+output_names[i]+'_accuracy'])))
    #loss
    plt.figure()
    for i in range(3):
        plt.plot(np.arange(0, epochs), history.history[output_names[i]+'_loss'], label='train_accuracy')
        plt.plot(np.arange(0, epochs), history.history['val_'+output_names[i]+'_loss'], label='val_accuracy')
        plt.title('Loss of '+model_types[i]+' for training dataset '+ str(dataset))
        plt.xlabel('Epoch #')
        plt.ylabel('Loss')
        plt.legend(loc='upper right')
        plt.show()
        print('Validation loss for {} is: {}'.format(model_types[i], max(history.history['val_'+output_names[i]+'_loss'])))
    dataset += 1
    

## Predict the test files for submission

In [ ]:
IMG_SIZE=64
N_CHANNELS=1
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_feather('/kaggle/input/bengaliaicv19feather/test_image_data_{}.feather'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)

    X_test = resize(df_test_img, need_progress_bar=False)/255
    X_test = X_test.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
    preds = model.predict(X_test)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test
    gc.collect()

df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample.head()